In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from scipy.stats import pearsonr
from collections import Counter

<IPython.core.display.Javascript object>

In [3]:
def convert_to_float(value):
    try:
        # If it's a string enclosed in single quotes, remove quotes and replace comma with dot
        if isinstance(value, str):
            return float(value.replace(",", "."))
        else:
            return float(value)
    except ValueError as e:
        print(e)
        return float("nan")

<IPython.core.display.Javascript object>

In [4]:
def preprocess_numeric_cols(df, columns):
    for col in columns:
        df[col] = df[col].astype(str)  # Ensure string type for string operations
        df[col] = df[col].str.replace("#", "")  # Remove '#' characters
        df[col] = df[col].str.replace(",", ".")  # Replace ',' with '.' for decimals
    return df

<IPython.core.display.Javascript object>

# Data preparation - 209 plant aa data

### Reading the files and extracting relevant information:

In [5]:
plant = "AA"
xls_files = {}

xls = pd.ExcelFile(
    f"../../../data/raw/209/EMBRAPII hubIC IACC_CIMENTO_{plant}.xlsx",
    engine="openpyxl",
)
xls_files[plant] = xls

<IPython.core.display.Javascript object>

In [6]:
print("Plant: ", plant, xls.sheet_names)

Plant:  AA ['INSTRUÇÕES', 'Clínquer', '95', '110', '133', '145']


<IPython.core.display.Javascript object>

In [7]:
cement_types_per_plant = []
CEMENT_TYPE_COLS = ["Tipo de cimento", "Classe de resistência"]
dataframes = []

for plant, xls in xls_files.items():
    for sheet_name in xls.sheet_names[2:]:
        df = pd.read_excel(xls_files[plant], sheet_name, header=[1, 2])

        if not df[CEMENT_TYPE_COLS].iloc[2:].isna().all().all():
            cement_type = (
                df[CEMENT_TYPE_COLS]
                .loc[2:]
                .astype(str)
                .sum(axis=1)
                .str.replace(" ", "")
                .str.replace("-", "")
                .str.replace("nan", "")
                .unique()
            )

        else:
            cement_type = (
                df[[("Obs.", "Unnamed: 8_level_1")]]
                .loc[2:]
                .astype(str)
                .sum(axis=1)
                .str.replace(" ", "")
                .str.replace("-", "")
                .unique()
            )

        cement_types_per_plant.append((plant, sheet_name, cement_type))
        dataframes.append(df)

<IPython.core.display.Javascript object>

In [8]:
cement_types_per_plant

[('AA', '95', array(['CPIII40'], dtype=object)),
 ('AA', '110', array(['CPINDCC'], dtype=object)),
 ('AA', '133', array(['CPIIIRS32'], dtype=object)),
 ('AA', '145', array(['CPIIIRS40'], dtype=object))]

<IPython.core.display.Javascript object>

In [9]:
cements = []
for tup in cement_types_per_plant:
    cements.append(tup[2][0])
Counter(cements)

Counter({'CPIII40': 1, 'CPINDCC': 1, 'CPIIIRS32': 1, 'CPIIIRS40': 1})

<IPython.core.display.Javascript object>

In [10]:
dataframes = []

for plant, xls in xls_files.items():
    for sheet_name in xls.sheet_names[2:]:
        df = pd.read_excel(xls_files[plant], sheet_name, header=[0, 1, 2])
        df["Unnamed: 0_level_0", "Unnamed: 0_level_1", "Unnamed: 0_level_2"] = plant
        dataframes.append(df.drop([0, 1], axis=0).reset_index(drop=True).copy())

<IPython.core.display.Javascript object>

In [11]:
df = pd.concat(dataframes, axis=0).reset_index(drop=True)

<IPython.core.display.Javascript object>

## Initial Preprocessing

In [12]:
df = pd.concat(
    [
        df["Unnamed: 0_level_0", "Unnamed: 0_level_1", "Unnamed: 0_level_2"],
        df["Dados iniciais"],
        df["Cimento"]["Composição"].rename(
            {"Sulfato de cálcio": "Sulfato de cálcio (composição)"}, axis=1
        ),
        df["Cimento"]["Análise mineralógica"],
        df["Cimento"]["Análise química"],
        df["Característias físicas do cimento"],
        df["Propriedades do cimento"],
    ],
    axis=1,
)

<IPython.core.display.Javascript object>

In [13]:
# df.info()

<IPython.core.display.Javascript object>

### 1. Dropping columns where either there is no data nor it contributes to the analysis
### 2. Renaming columns to a short identifiable name

In [14]:
COLUMNS_TO_DROP = [
    ("Data", "Medida"),
    ("Horário", "Produção"),
    ("Horário", "Medida"),
    ("Obs.", "Unnamed: 148_level_2"),
    ("Tipo de cimento", "Unnamed: 6_level_2"),
    ("Classe de resistência", "Unnamed: 7_level_2"),
    ("Obs.", "Unnamed: 8_level_2"),
    "Obs.",
    ("Obs.", "Unnamed: 148_level_2"),
    ("Unnamed: 141_level_1", "Obs."),
    # ("Unnamed: 139_level_1", "#400"),
]

COLUMNS_TO_RENAME = {
    ("Unnamed: 0_level_0", "Unnamed: 0_level_1", "Unnamed: 0_level_2"): "Plant",
    ("Cidade", "Unnamed: 1_level_2"): "City",
    ("Data", "Produção"): "Date",  # Arrumar!
    ("Unnamed: 136_level_1", "Blaine"): "Blaine",
    ("Unnamed: 137_level_1", "#200"): "#200",
    ("Unnamed: 138_level_1", "#325"): "#325",
    ("Unnamed: 139_level_1", "#400"): "#400",
    ("Unnamed: 140_level_1", "Massa específica"): "Specific Gravity",
    ("Tempo de pega", "Inicio"): "Initial setting time",
    ("Tempo de pega", "Fim"): "Final setting time",
    ("Resistência à compressão", "1 d"): "CS1",
    ("Resistência à compressão", "3 d"): "CS3",
    ("Resistência à compressão", "7 d"): "CS7",
    ("Resistência à compressão", "28 d"): "CS28",
    "Perda ao fogo": "Loss on Ignition",
    "Resíduo insolúvel": "Insoluble Residue",
    "Clínquer": "Clinker",
    "Fíler calcário": "Calcareous filler",
    "Cinza volante": "Fly ash",
    "Escória": "Slag",
    "Argila calcinada": "Calcined clay",
    "Sulfato de cálcio": "Calcium sulfate",
    "C3S total": "Total C3S",
    "C2S total": "Total C2S",
    "C2S alpha": "Alpha C2S",
    "C2S beta": "Beta C2S",
    "C2S gama": "Gamma C2S",
    "C4AF": "C4AF",
    "C3A": "C3A",
    "C3A cubic": "Cubic C3A",
    "C3A orto": "Orthorhombic C3A",
    "CaO livre": "Free CaO",
    "Portlandita": "Portlandite",
    "Periclasio": "Periclase",
    "Arcanita": "Arcanite",
    "Aphthalita": "Aphthitalite",
    "Langbeinita": "Langbeinite",
    "Bassanita": "Bassanite",
    "Anidrita": "Anhydrite",
    "Calcita": "Calcite",
    "Dolomita": "Dolomite",
    "Quartzo": "Quartz",
    "Muscovita": "Muscovite",
    "Sulfato de cálcio (composição)": "Calcium sulfate (composition)",
}

<IPython.core.display.Javascript object>

### Defining a single variable to identify Cement Type

In [15]:
# Single variable for the Cement Type
df["Cement_Type"] = (
    df[
        [
            ("Tipo de cimento", "Unnamed: 6_level_2"),
            ("Classe de resistência", "Unnamed: 7_level_2"),
            ("Obs.", "Unnamed: 8_level_2"),
        ]
    ]
    .fillna("")
    .astype(str)
    .sum(axis=1)
)

# # Preprocessing
df["Cement_Type"] = df["Cement_Type"].replace("Fibro", "Fibrocimento")

# Preprocessing
df["Cement_Type"] = (
    df["Cement_Type"]
    .str.replace("Expedido", "")
    .str.replace("Linha 9", "")
    .str.replace("Itajaí", "")
    .str.replace("D-G", "")
    .str.replace("D-E", "")
    .str.replace("DF2", "")
    .str.replace("CPIIF40", "CP II-F-40")
    .str.replace("CPIIF40", "CP II-F40")
    .str.replace("CP VARI", "CP V-ARI")
    .str.replace("CPVARI", "CP V-ARI")
    .str.replace("CP V-RSARI", "CP V-ARI RS")
    .str.replace("CP III-RS40", "CP III-40 RS")
    .str.replace("CP III-RS32", "CP III-32 RS")
    .str.replace("CP IV-RS32", "CP IV-32 RS")
    .str.replace("CP III40", "CP III-40")
    .str.replace("CP III32", "CP III-32")
    .str.replace("CP I-S40", "CP I-S-40")
    .str.replace("CP I40", "CP I-40")
    .str.replace("CP II-E32", "CP II-E-32")
    .str.replace("CP II-E40", "CP II-E-40")
    .str.replace("CP II-F FIBRO40", "CP II-F-40")
    .str.replace("CP II-F32", "CP II-F-32")
    .str.replace("CP II-F40", "CP II-F-40")
    .str.replace("CP II-Z32", "CP II-Z-32")
    .str.replace("CP II-Z40", "CP II-Z-40")
    .str.replace("CP IV32", "CP IV-32")
    .str.replace("CP IND", "CP I")
    .str.replace("CPINDCC", "CP I")
    .str.strip()
)

df = df.drop(COLUMNS_TO_DROP, axis=1).rename(COLUMNS_TO_RENAME, axis=1).copy()
df = df.drop([0]).reset_index(drop=True)
df = df.infer_objects()

<IPython.core.display.Javascript object>

### Dropping and rename the columns defined above

### Cleaning and converting numeric columns to float

In [16]:
NUMERIC_COLUMNS = [
    "Calcium sulfate (composition)",
    "Clinker",
    "Calcareous filler",
    "Fly ash",
    "Slag",
    "Calcined clay",
    "Calcium sulfate",
    "Total C3S",
    "Total C2S",
    "Alpha C2S",
    "Beta C2S",
    "Gamma C2S",
    "C4AF",
    "C3A",
    "Cubic C3A",
    "Orthorhombic C3A",
    "Free CaO",
    "Portlandite",
    "Periclase",
    "Arcanite",
    "Aphthitalite",
    "Langbeinite",
    "Bassanite",
    "Anhydrite",
    "Calcite",
    "Dolimita",
    "Quartz",
    "Muscovite",
    "CaO",
    "MgO",
    "Na2O",
    "Al2O3",
    "SiO2",
    "SO3",
    "K2O",
    "TiO2",
    "Fe2O3",
    "Loss on Ignition",
    "Insoluble Residue",
    "Blaine",
    "#200",
    "#325",
    "#400",
    "Specific Gravity",
    "Final setting time",
    "Initial setting time",
    "CS1",
    "CS3",
    "CS7",
    "CS28",
]

df = preprocess_numeric_cols(df, NUMERIC_COLUMNS)

# Preprocessing to fix numeric columns
df[NUMERIC_COLUMNS] = df[NUMERIC_COLUMNS].map(convert_to_float)

# Converting Date to pandas datetime
df["Date"] = pd.to_datetime(df["Date"])

# Drop rows completely missing
index_to_keep = (
    df.drop(["Plant", "Date", "Cement_Type"], axis=1).dropna(axis=0, how="all").index
)
df = df.iloc[index_to_keep].reset_index(drop=True)

df = df.infer_objects()

could not convert string to float: '-'
could not convert string to float: '-'
could not convert string to float: '-'
could not convert string to float: '-'
could not convert string to float: '-'
could not convert string to float: '-'
could not convert string to float: '-'
could not convert string to float: '-'
could not convert string to float: '-'
could not convert string to float: '-'
could not convert string to float: '-'
could not convert string to float: '-'
could not convert string to float: '-'
could not convert string to float: '-'
could not convert string to float: '-'
could not convert string to float: '-'
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string to float: ' '
could not convert string 

<IPython.core.display.Javascript object>

## Dropping missing features

### Drop rows completely missing

In [17]:
index_to_keep = (
    df.drop(["Plant", "City", "Date", "Cement_Type"], axis=1)
    .dropna(axis=0, how="all")
    .index
)
df = df.iloc[index_to_keep].reset_index(drop=True)

<IPython.core.display.Javascript object>

### Removing features
 that will be fitted on this data.

1. Removing features with features having ~50% or more of missing values
2. Removing features with zero variance
3. Removing features with more than 70% of zeros

In [18]:
(df.isna().sum() / df.shape[0] * 100).sort_values(ascending=False).to_frame(
    name="Missing (%)"
).style.background_gradient(cmap="Reds")

,Missing (%)
Alpha C2S,100.000000
Beta C2S,100.000000
CS1,100.000000
Clinker,100.000000
Calcareous filler,100.000000
Fly ash,100.000000
Slag,100.000000
Calcined clay,100.000000
Anhydrite,100.000000
Specific Gravity,100.000000


<IPython.core.display.Javascript object>

<h3>Percentage of missing values:</h3>
<table>
    <th>Plant A - Full Dataset:</th>
    <th></th>
    <tr>
        <td>Alpha C2S</td>
        <td>100.00</td>
    </tr>
    <tr>
        <td>Beta C2S</td>
        <td>100.00</td>
    </tr>
    <tr>
        <td>CS1</td>
        <td>100.00</td>
    </tr>  
    <tr>
        <td>Clinker</td>
        <td>100.00</td>
    </tr>
    <tr>
        <td>Calcareous filler</td>
        <td>100.00</td>
    </tr>
    <tr>
        <td>Fly ash</td>
        <td>100.00</td>
    </tr>
    <tr>
        <td>Slag</td>
        <td>100.00</td>
    </tr>  
    <tr>
        <td>Calcined clay</td>
        <td>100.00</td>
    </tr>
    <tr>
        <td>Anhydrite</td>
        <td>100.00</td>>
    </tr>
    <tr>
        <td>Specific Gravity</td>
        <td>100.00</td>
    </tr>
    <tr>
        <td>Gamma C2S</td>
        <td>100.00</td>
    </tr>  
    <tr>
        <td>Calcium sulfate (composition)</td>
        <td>84.50</td>
    </tr>
    <tr>
        <td>Calcium sulfate</td>
        <td>65.10</td>
    </tr>
    <tr>
        <td>"Total C2S"</td>
        <td>65.10</td>
    </tr>
    <tr>
        <td>#325</td>
        <td>64.87</td>
    </tr>
    <tr>
        <td>Total C3S</td>
        <td>51.09</td>
    </tr>
    <tr>
        <td>Langbeinite</td>
        <td>49.60</td>
    </tr>
</table>

In [19]:
FEATRUES_TO_DROP = [
    "Alpha C2S",
    "Beta C2S",
    "CS1",
    "Clinker",
    "Calcareous filler",
    "Fly ash",
    "Slag",
    "Calcined clay",
    "Anhydrite",
    "Specific Gravity",
    "Gamma C2S",
    "Calcium sulfate (composition)",
    "Calcium sulfate",
    "Total C2S",
    "#325",
    "Total C3S",
    "Langbeinite",
]
df = df.drop(labels=FEATRUES_TO_DROP, axis=1)

<IPython.core.display.Javascript object>

### Removing features with more than 23% of zeros

In [20]:
zero_values = {}
for col in df.select_dtypes(include="number").columns:
    zero_percentages = (df[df[col].eq(0)].shape[0] / df.shape[0]) * 100
    zero_values[col] = zero_percentages

zero_percentages = pd.Series(zero_values, name=f"Zero (%) for {plant}")
zero_percentages = zero_percentages.sort_values(ascending=False)
zero_percentages = zero_percentages.to_frame(name=f"Zero (%) for {plant}")
zero_percentages.style.background_gradient(cmap="Reds")

,Zero (%) for AA
Dolimita,77.554535
Arcanite,55.223881
Free CaO,32.261768
#200,23.076923
Aphthitalite,11.481056
Muscovite,0.287026
Bassanite,0.114811
Portlandite,0.057405
Periclase,0.057405
Quartz,0.057405


<IPython.core.display.Javascript object>

<h3>Percentage of zero values:</h3>
<table>
    <th>Plant A - Full Dataset:</th>
    <th></th>
    <tr>
        <td>Dolimita</td>
        <td>77.55</td>
    </tr>
    <tr>
        <td>Arcanite</td>
        <td>55.22</td>
    </tr>
    <tr>
        <td>Free CaO</td>
        <td>32.26</td>
    </tr>  
    <tr>
        <td>#200</td>
        <td>23.10</td>
    </tr>
    <tr>
        <td>Calcareous filler</td>
        <td>100.00</td>
    </tr>
</table>

In [21]:
FEATRUES_TO_DROP = ["Dolimita", "Arcanite", "Free CaO", "#200"]
df = df.drop(labels=FEATRUES_TO_DROP, axis=1)

<IPython.core.display.Javascript object>

### Dropping variable with City and Plant identification

In [22]:
FEATRUES_TO_DROP = ["City", "Plant"]
df = df.drop(labels=FEATRUES_TO_DROP, axis=1)

<IPython.core.display.Javascript object>

### Dropping rows where the target variable is null

In [23]:
df = df[df["CS28"].notna()]

<IPython.core.display.Javascript object>

### Dropping rows where the target variable is zero

In [24]:
df = df[~df["CS28"].eq(0)]

<IPython.core.display.Javascript object>

In [25]:
df.shape

(1720, 31)

<IPython.core.display.Javascript object>

In [26]:
# df.describe().T

<IPython.core.display.Javascript object>

In [27]:
# df_copy = df.copy()

<IPython.core.display.Javascript object>

### Dropping rows where any other variable has a zero value

In [28]:
for col in df.columns:
    df = df[~df[col].eq(0)]

<IPython.core.display.Javascript object>

In [29]:
df.shape

(1511, 31)

<IPython.core.display.Javascript object>

In [30]:
(df.isna().sum() / df.shape[0] * 100).sort_values(ascending=False).to_frame(
    name="Missing (%)"
).style.background_gradient(cmap="Reds")

,Missing (%)
C3A,38.385175
Aphthitalite,37.524818
Quartz,19.788220
Muscovite,19.788220
Cubic C3A,19.788220
Orthorhombic C3A,19.788220
Portlandite,19.788220
Periclase,19.788220
Bassanite,19.788220
Calcite,19.788220


<IPython.core.display.Javascript object>

## Dropping rows with increased missing values after preprocessing

<h3>Percentage of missing values:</h3>
<table>
    <th>Plant A - Full Dataset:</th>
    <th></th>
    <tr>
        <td>C3A</td>
        <td>38.39</td>
    </tr>
    <tr>
        <td>Aphthitalite</td>
        <td>37.52</td>
    </tr>
</table>

In [31]:
FEATRUES_TO_DROP = ["C3A", "Aphthitalite"]
df = df.drop(labels=FEATRUES_TO_DROP, axis=1)

<IPython.core.display.Javascript object>

In [32]:
(df.isna().sum() / df.shape[0] * 100).sort_values(ascending=False).to_frame(
    name="Missing (%)"
).style.background_gradient(cmap="Reds")

,Missing (%)
Muscovite,19.788220
C4AF,19.788220
Cubic C3A,19.788220
Orthorhombic C3A,19.788220
Portlandite,19.788220
Periclase,19.788220
Bassanite,19.788220
Calcite,19.788220
Quartz,19.788220
Insoluble Residue,0.926539


<IPython.core.display.Javascript object>

In [33]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
Date,1511,2022-04-24 05:36:24.778292480,2021-01-04 00:00:00,2021-10-06 00:00:00,2022-04-29 00:00:00,2022-11-23 00:00:00,2023-07-17 00:00:00,NaN
C4AF,1212.0,6.661262,2.19,4.27,5.91,9.3,11.56,2.743028
Cubic C3A,1212.0,1.952434,0.46,1.04,1.87,2.57,7.55,1.135169
Orthorhombic C3A,1212.0,0.397426,0.01,0.01,0.08,0.77,2.28,0.506878
Portlandite,1212.0,1.081378,0.08,0.55,0.945,1.47,4.29,0.681564
Periclase,1212.0,0.394035,0.01,0.22,0.35,0.54,1.76,0.226943
Bassanite,1212.0,1.224125,0.15,0.7675,1.1,1.63,3.56,0.591011
Calcite,1212.0,3.639909,0.95,2.35,3.365,5.0625,9.8,1.568483
Quartz,1212.0,0.66717,0.03,0.33,0.6,0.92,6.58,0.453711
Muscovite,1212.0,0.213812,0.01,0.09,0.18,0.32,0.54,0.147779


<IPython.core.display.Javascript object>

In [34]:
df.describe(exclude="number").T

,count,unique,top,freq,mean,min,25%,50%,75%,max
Date,1511,NaN,NaN,NaN,2022-04-24 05:36:24.778292480,2021-01-04 00:00:00,2021-10-06 00:00:00,2022-04-29 00:00:00,2022-11-23 00:00:00,2023-07-17 00:00:00
Cement_Type,1511,4,CP I,605,NaN,NaN,NaN,NaN,NaN,NaN


<IPython.core.display.Javascript object>

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1511 entries, 0 to 1730
Data columns (total 29 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Date                  1511 non-null   datetime64[ns]
 1   C4AF                  1212 non-null   float64       
 2   Cubic C3A             1212 non-null   float64       
 3   Orthorhombic C3A      1212 non-null   float64       
 4   Portlandite           1212 non-null   float64       
 5   Periclase             1212 non-null   float64       
 6   Bassanite             1212 non-null   float64       
 7   Calcite               1212 non-null   float64       
 8   Quartz                1212 non-null   float64       
 9   Muscovite             1212 non-null   float64       
 10  CaO                   1511 non-null   float64       
 11  MgO                   1511 non-null   float64       
 12  Na2O                  1511 non-null   float64       
 13  Al2O3                 1

<IPython.core.display.Javascript object>

### Sort the dataset by date

In [36]:
# We first make sure Date column was infered correctly
df["Date"].dtype

dtype('<M8[ns]')

<IPython.core.display.Javascript object>

In [37]:
df = df.sort_values(by="Date")

<IPython.core.display.Javascript object>

## Saving the dataset

<h3>Saving for feature engineering</h3>

In [38]:
df.to_csv("../../../data/interim/209/aa.csv", index=False)

<IPython.core.display.Javascript object>